<a href="https://colab.research.google.com/github/nishanadahal/skills-introduction-to-github/blob/main/FinanceAPIHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Task 1:  Fetch Basic Stock Data


import json
import pandas as pd
import json
import requests

stocks = input("Please input two or more stock symbols (comma separated, no space): ").upper()
apikey = ""

url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":stocks}

headers = {
    'x-api-key': apikey
    }

response = requests.request("GET", url, headers=headers, params=querystring)
if response.status_code == 200:
  df = pd.DataFrame(response.json()['quoteResponse']['result'])
  print(df[['symbol', 'displayName', 'regularMarketPrice']])
else:
  print("Error: " + str(response.status_code))

In [ ]:
#Task 2: Fetch additional stock data

#2.1:

stock = input("Enter stock symbol: ").upper()
module = input("Choose a module (financialData for ROA, summaryDetail for 52-week high/low): ").strip()
apikey = ""
headers = {
    'x-api-key': apikey
}

if module not in ["financialData", "summaryDetail"]:
    print("Invalid module. Choose 'financialData' or 'summaryDetail'.")
else:
    url = f"https://yfapi.net/v11/finance/quoteSummary/{stock}"
    querystring = {"modules": module, "symbols": stock}

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json().get('quoteSummary', {}).get('result', [{}])[0]

        if module == "financialData":
            df = pd.DataFrame([data['financialData']])
            print(df[['returnOnAssets']])
        elif module == "summaryDetail":
            df = pd.DataFrame([data['summaryDetail']])
            print(df[['fiftyTwoWeekLow', 'fiftyTwoWeekHigh']])
    else:
        print("Error getting response")



In [ ]:
#2.2 Fetch additional data using modules (trending)

apikey = ""
headers = {'x-api-key': apikey}

url = "https://yfapi.net/v1/finance/trending/US"
response = requests.get(url, headers=headers)

if response.status_code == 200:
    t_data = response.json().get('finance', {}).get('result', [{}])[0].get('quotes', [])

    if t_data:
        tickers = [item.get('symbol', 'N/A') for item in t_data]

        for symbol in tickers:
            details_url = f"https://yfapi.net/v6/finance/quote"
            querystring = {"symbols": symbol}

            response_details = requests.get(details_url, headers=headers, params=querystring)

            if response_details.status_code == 200:
                stock_data = response_details.json().get('quoteResponse', {}).get('result', [{}])[0]

                ticker = stock_data.get("symbol", "N/A")
                name = stock_data.get("longName", stock_data.get("shortName", "N/A"))
                price = stock_data.get("regularMarketPrice", "N/A")
                high_52 = stock_data.get("fiftyTwoWeekHigh", "N/A")
                low_52 = stock_data.get("fiftyTwoWeekLow", "N/A")
                print("Trending stocks:")
                print(f"Symbol: {ticker}, Display Name: {name}, Current Market Price: {price}, 52-Week High: {high_52}, 52-Week Low: {low_52}")
            else:
                print(f"Error fetching details for {symbol}")
    else:
        print("No trending stocks found.")
else:
    print("Error fetching trending stocks")
